In [1]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [2]:
import numpy as np
from keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import pad_sequences,to_categorical

In [3]:
corpus="""Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do:
once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it,
'and what is the use of a book,' thought Alice 'without pictures or conversation?'"""

corpus

"Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do:\nonce or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it,\n'and what is the use of a book,' thought Alice 'without pictures or conversation?'"

In [4]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words=len(tokenizer.word_index)+1

total_words  #only unique words

44

In [5]:
input_seq=[]
words=corpus.lower().split()
len(words)

57

In [6]:
for i in range(1,len(words)):
    n_gram = words[:i+1]
    token_list=tokenizer.texts_to_sequences([" ".join(n_gram)])[0]
    input_seq.append(token_list)
    print(n_gram)  #for reference

input_seq

['alice', 'was']
['alice', 'was', 'beginning']
['alice', 'was', 'beginning', 'to']
['alice', 'was', 'beginning', 'to', 'get']
['alice', 'was', 'beginning', 'to', 'get', 'very']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the']
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', 'bank,']
['alic

[[4, 5],
 [4, 5, 14],
 [4, 5, 14, 6],
 [4, 5, 14, 6, 15],
 [4, 5, 14, 6, 15, 16],
 [4, 5, 14, 6, 15, 16, 17],
 [4, 5, 14, 6, 15, 16, 17, 1],
 [4, 5, 14, 6, 15, 16, 17, 1, 18],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2, 21],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2, 21, 22],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2, 21, 22, 1],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2, 21, 22, 1, 23],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2, 21, 22, 1, 23, 24],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2, 21, 22, 1, 23, 24, 6],
 [4, 5, 14, 6, 15, 16, 17, 1, 18, 19, 7, 8, 20, 2, 21, 22, 1, 23, 24, 6, 25],
 [4,
  5,
  14,
  6,
  15,
  16,
  17,
  1,
  18,
  19,
  7,
  8,
  20,
  2,
  21,
  22,
  1,
  23,
  24,
  6,
  25,
  26],
 [4,

In [7]:
max_seq_len = max([len(seq) for seq in input_seq])
input_seq=np.array(pad_sequences(input_seq,maxlen=max_seq_len,padding='pre'))

In [8]:
x=input_seq[:,:-1]
y=input_seq[:,-1]
y

#from keras.utils import to_categorical
#y = to_categorical(y, num_classes=total_words)
#use binary_crossentropy for this

array([ 5, 14,  6, 15, 16, 17,  1, 18, 19,  7,  8, 20,  2, 21, 22,  1, 23,
       24,  6, 25, 26,  3, 27, 28,  9, 29, 30,  2, 10,  7,  8,  5, 31, 32,
       11,  9, 33, 12,  3, 34, 35, 11, 36, 37, 38,  2, 39,  1, 40, 13, 41,
        4, 42, 12,  3, 13])

In [9]:
model=Sequential()
model.add(Embedding(total_words, 50, input_length=max_seq_len - 1))
model.add(LSTM(150))
model.add(Dense(total_words,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

C:\Users\naras\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(x, y, epochs=200, verbose=1)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.0223 - loss: 3.7860
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0908 - loss: 3.7789
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1473 - loss: 3.7718
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1592 - loss: 3.7653
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1473 - loss: 3.7557
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.1235 - loss: 3.7392
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.0580 - loss: 3.7081
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.1116 - loss: 3.6559
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.0908 - loss: 3.5786
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0908 - loss: 3.5581
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0893 - loss: 3.4783
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.1012 - lo

In [12]:
def generate_text(user_ip,next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([user_ip])[0]
        token_list = pad_sequences([token_list],maxlen=max_seq_len-1,padding='pre')
        pred = np.argmax(model.predict(token_list), axis=-1)[0]

        for word,index in tokenizer.word_index.items():
            if index==pred:
                user_ip+=" "+word
                break

        return user_ip                                       

In [30]:
user_input = input("Enter a starting word or phrase: ")
print("Generated text:\n", generate_text(user_input, next_words=20))

Enter a starting word or phrase:  having


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Generated text:
 having was
